In [2]:
from resinkit import AsyncResinkitClient
import asyncio
import nest_asyncio
nest_asyncio.apply()  # Allow nested event loops

rc = AsyncResinkitClient(flink_gateway_url="http://localhost:8083")
res = asyncio.run(rc.query("SELECT 23 as age, 'Alice Liddel' as name;"))


In [3]:
res

,age,name
0,23,Alice Liddel
